# Running Marlowe Contracts with the Plutus Application Backend (PAB)



## Other Lectures

*   Overview of Marlowe CLI
*   Installing Marlowe CLI and Associated Tools~
*   Running Marlowe Contracts without Blockchain Transactions
*   Running Marlowe Contacts on the Blockchain
*   ~Running Marlowe Contracts with the Plutus Application Backend (PAB)~
*   Reference for Marlowe CLI Commands